# Import the necessary libraries

In [1]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import pandas as pd
 
# Configure logging
logging.basicConfig(filename='../logs/',
                    level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
  

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from load_csv_data import Load_CSV_Data


# Load the data sets

In [2]:
credit_df = Load_CSV_Data('../data/cleaned_credit_card_data.csv')
credit_df.load_csv_data()
credit_df = credit_df.get_data()

Data successfully loaded from ../data/cleaned_credit_card_data.csv


In [3]:
fraud_df = Load_CSV_Data('../data/cleaned_fraud_data.csv')
fraud_df.load_csv_data()
fraud_df = fraud_df.get_data()

Data successfully loaded from ../data/cleaned_fraud_data.csv


In [4]:
ip_df = Load_CSV_Data('../data/cleaned_IpAddress_to_Country.csv')
ip_df.load_csv_data()
ip_df = ip_df.get_data()

Data successfully loaded from ../data/cleaned_IpAddress_to_Country.csv


# Merge the data sets

In [5]:
# Convert the 'ip_address' in df_user_cleaned to integer (if not already)
fraud_df['ip_address'] = fraud_df['ip_address'].astype(float)

# Merge fraud data with IP address geolocation data based on IP address range
def merge_ip_geolocation(fraud_df, ip_df):
    """
    Merges fraud dataset with IP geolocation dataset.
    
    Parameters:
    fraud_df (pd.DataFrame): Fraud dataset with 'ip_address' column.
    ip_df (pd.DataFrame): IP geolocation dataset with 'lower_bound_ip_address' and 'upper_bound_ip_address' columns.
    
    Returns:
    pd.DataFrame: Merged dataset.
    """
    merged_data = pd.merge_asof(
        fraud_df.sort_values('ip_address'),  # Sort fraud data by IP
        ip_df.sort_values('lower_bound_ip_address'),  # Sort IP data by lower bound
        left_on='ip_address',  # Merge on ip_address from fraud data
        right_on='lower_bound_ip_address',  # Merge on lower_bound_ip_address from IP geolocation data
        direction='backward'  # Merge backward to match within the IP range
    )

    # Filter to keep only the rows where the IP is within the upper bound
    merged_data = merged_data[merged_data['ip_address'] <= merged_data['upper_bound_ip_address']]
    
    return merged_data

# Apply the merging function
df_merged = merge_ip_geolocation(fraud_df, ip_df)

df_merged.head()

# Save the merged data
df_merged.to_csv("../data/merged_fraud_geolocation_data.csv", index=False)
print("Merged dataset saved as 'merged_fraud_geolocation_data.csv'")

Merged dataset saved as 'merged_fraud_geolocation_data.csv'


# Feature Engineering 

In [6]:
import pandas as pd

# Load the merged data
df_merged = pd.read_csv("../data/merged_fraud_geolocation_data.csv")

# Convert 'signup_time' and 'purchase_time' to datetime
df_merged['signup_time'] = pd.to_datetime(df_merged['signup_time'])
df_merged['purchase_time'] = pd.to_datetime(df_merged['purchase_time'])

# Feature 1: Transaction frequency and velocity (time between signup and purchase)
df_merged['transaction_velocity'] = (df_merged['purchase_time'] - df_merged['signup_time']).dt.total_seconds()

# Feature 2: Time-based features (hour of day and day of week)
df_merged['purchase_hour'] = df_merged['purchase_time'].dt.hour
df_merged['purchase_day_of_week'] = df_merged['purchase_time'].dt.dayofweek

# Save the feature-engineered data
df_merged.to_csv("../data/feature_engineered_data.csv", index=False)
print("Feature-engineered data saved as 'feature_engineered_data.csv'")


Feature-engineered data saved as 'feature_engineered_data.csv'


In [7]:
df_merged.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,lower_bound_ip_address,upper_bound_ip_address,country,transaction_velocity,purchase_hour,purchase_day_of_week
0,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47,KIXYSVCHIPQBR,SEO,Safari,F,30,1.677886e+07,0,16778240.0,16779263.0,Australia,3632215.0,3,6
1,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15,PKYOWQKWGJNJI,SEO,Chrome,F,34,1.684205e+07,0,16809984.0,16842751.0,Thailand,1231637.0,20,2
2,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44,LVCSXLISZHVUO,Ads,IE,M,29,1.684366e+07,0,16843264.0,16843775.0,China,1997534.0,23,5
3,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,UHAUHNXXUADJE,Direct,Chrome,F,30,1.693873e+07,0,16924672.0,16941055.0,China,7640897.0,16,5
4,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,XPGPMOHIDRMGE,SEO,Chrome,F,37,1.697198e+07,0,16941056.0,16973823.0,Thailand,10249579.0,4,1


# Encoding categorical column

In [9]:
import pandas as pd

# Load your dataset
df = pd.read_csv("../data/feature_engineered_data.csv")

# Apply frequency encoding to 'country', 'browser', and 'source'
frequency_encoded_columns = ['country', 'browser', 'source']
for col in frequency_encoded_columns:
    df[f"{col}_encoded"] = df[col].map(df[col].value_counts())

# One-hot encode 'sex' without causing multiple columns
df_encoded = pd.get_dummies(df, columns=['sex'], drop_first=True)

# Drop the original 'country', 'browser', and 'source' columns
df_encoded = df_encoded.drop(frequency_encoded_columns, axis=1)

# Check the first few rows of the encoded dataset
df_encoded.head()


,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,lower_bound_ip_address,upper_bound_ip_address,transaction_velocity,purchase_hour,purchase_day_of_week,country_encoded,browser_encoded,source_encoded,sex_M
0,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47,KIXYSVCHIPQBR,30,1.677886e+07,0,16778240.0,16779263.0,3632215.0,3,6,1844,21074,51960,False
1,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15,PKYOWQKWGJNJI,34,1.684205e+07,0,16809984.0,16842751.0,1231637.0,20,2,291,52560,51960,False
2,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44,LVCSXLISZHVUO,29,1.684366e+07,0,16843264.0,16843775.0,1997534.0,23,5,12038,31364,51067,True
3,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,UHAUHNXXUADJE,30,1.693873e+07,0,16924672.0,16941055.0,7640897.0,16,5,12038,52560,26119,False
4,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,XPGPMOHIDRMGE,37,1.697198e+07,0,16941056.0,16973823.0,10249579.0,4,1,291,52560,51960,False


# Encode device id

In [10]:
import category_encoders as ce

# Initialize the Target Encoder as an alternative
target_encoder = ce.TargetEncoder(cols=['device_id'])

# Fit and transform the data
df_encoded['device_id_target_encoded'] = target_encoder.fit_transform(df_encoded['device_id'], df_encoded['class'])

# Drop the original device_id column
fraud_data = df_encoded.drop(columns=['device_id'])

# Check the result
fraud_data.head()


,user_id,signup_time,purchase_time,purchase_value,age,ip_address,class,lower_bound_ip_address,upper_bound_ip_address,transaction_velocity,purchase_hour,purchase_day_of_week,country_encoded,browser_encoded,source_encoded,sex_M,device_id_target_encoded
0,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47,30,1.677886e+07,0,16778240.0,16779263.0,3632215.0,3,6,1844,21074,51960,False,0.082634
1,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15,34,1.684205e+07,0,16809984.0,16842751.0,1231637.0,20,2,291,52560,51960,False,0.082634
2,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44,29,1.684366e+07,0,16843264.0,16843775.0,1997534.0,23,5,12038,31364,51067,True,0.081518
3,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,30,1.693873e+07,0,16924672.0,16941055.0,7640897.0,16,5,12038,52560,26119,False,0.082634
4,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,37,1.697198e+07,0,16941056.0,16973823.0,10249579.0,4,1,291,52560,51960,False,0.082634


# Handling Datetime

In [11]:
# Convert 'signup_time' and 'purchase_time' to datetime format
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Create new features by extracting time differences and useful components
fraud_data['signup_year'] = fraud_data['signup_time'].dt.year
fraud_data['signup_month'] = fraud_data['signup_time'].dt.month
fraud_data['signup_day'] = fraud_data['signup_time'].dt.day
fraud_data['signup_hour'] = fraud_data['signup_time'].dt.hour

fraud_data['purchase_year'] = fraud_data['purchase_time'].dt.year
fraud_data['purchase_month'] = fraud_data['purchase_time'].dt.month
fraud_data['purchase_day'] = fraud_data['purchase_time'].dt.day
fraud_data['purchase_hour'] = fraud_data['purchase_time'].dt.hour

# Create a feature for the time difference between signup and purchase (in hours)
fraud_data['time_diff_hours'] = (fraud_data['purchase_time'] - fraud_data['signup_time']).dt.total_seconds() / 3600

# Now drop the original datetime columns
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])

# Continue with feature-target separation
X_fraud = fraud_data.drop(columns=['class'])
y_fraud = fraud_data['class']


In [13]:
fraud_data.head()


,user_id,purchase_value,age,ip_address,class,lower_bound_ip_address,upper_bound_ip_address,transaction_velocity,purchase_hour,purchase_day_of_week,...,sex_M,device_id_target_encoded,signup_year,signup_month,signup_day,signup_hour,purchase_year,purchase_month,purchase_day,time_diff_hours
0,247547,47,30,1.677886e+07,0,16778240.0,16779263.0,3632215.0,3,6,...,False,0.082634,2015,6,28,3,2015,8,9,1008.948611
1,220737,15,34,1.684205e+07,0,16809984.0,16842751.0,1231637.0,20,2,...,False,0.082634,2015,1,28,14,2015,2,11,342.121389
2,390400,44,29,1.684366e+07,0,16843264.0,16843775.0,1997534.0,23,5,...,True,0.081518,2015,3,19,20,2015,4,11,554.870556
3,69592,55,30,1.693873e+07,0,16924672.0,16941055.0,7640897.0,16,5,...,False,0.082634,2015,2,24,6,2015,5,23,2122.471389
4,174987,51,37,1.697198e+07,0,16941056.0,16973823.0,10249579.0,4,1,...,False,0.082634,2015,7,7,12,2015,11,3,2847.105278


In [15]:
# Save the encoded dataset
fraud_data.to_csv("../data/encoded_fraud_data.csv", index=False)
print("Categorical features encoded and saved as 'encoded_fraud_data.csv'")

Categorical features encoded and saved as 'encoded_fraud_data.csv'
